In [1]:
import pandas as pd
import numpy as np

## KNN(K-nearest neighbor)

In [2]:
fish=pd.read_csv("http://bit.ly/fish_csv_data")

* 시각화 및 이상치 확인은 Logit 파일에서 실시했었음

In [14]:
import sklearn.preprocessing as skpre
import sklearn.model_selection as skmod

In [11]:
fish[fish.columns.difference(["Species"])]=pd.DataFrame(skpre.minmax_scale(fish[fish.columns.difference(["Species"])])).rename(columns=dict(zip(range(5),fish.columns.difference(["Species"]))))

In [16]:
train,test=skmod.train_test_split(fish,test_size=0.2,random_state=10,shuffle=True,stratify=fish["Species"])

In [18]:
train["Species"].value_counts()

Species
Perch        45
Bream        28
Roach        16
Pike         13
Smelt        11
Parkki        9
Whitefish     5
Name: count, dtype: int64

In [19]:
test["Species"].value_counts()

Species
Perch        11
Bream         7
Pike          4
Roach         4
Smelt         3
Parkki        2
Whitefish     1
Name: count, dtype: int64

-----------

### KNN 원리
- 우리는 데이터의 분류의 기준을 `데이터간의 거리`로 하기때문에 이에 대한 이해가 반드시 필요하다         
대표적인 두개의 data 거리를 측정하는 방법을 알아보자

- 유클리디안 거리

$\text{distance}_{\text{euclidean}}(P, Q) = \sqrt{\sum_{i=1}^{n}(P_i - Q_i)^2}$

- 맨해튼거리

$\text{distance}_{\text{manhattan}}(P, Q) = \sum_{i=1}^{n}|P_i - Q_i|$

- 민코프스키 거리

$\text{distance}_{\text{Minkowski}}(P, Q) = \left(\sum_{i=1}^{n}|P_i - Q_i|^p\right)^{\frac{1}{p}}$

- 체비쇼프 거리
> 군집간의 최대거리를 측정할때 유용하게 쓰임

$\text{distance}_{\text{Chebyshev}}(P, Q) = \max(|P_i - Q_i|)$

* 코사인 거리

$\text{distance}_{\text{cosine}}(A, B) = 1 - \frac{\sum_{i=1}^{n} (A_i \cdot B_i)}{\sqrt{\sum_{i=1}^{n} A_i^2} \cdot \sqrt{\sum_{i=1}^{n} B_i^2}}$

-----------------

In [13]:
import sklearn.neighbors as sknei

#### 참고
- metric(수학에서 거리라는 말이다) : default=민코브스키
- int : 민코브스키 거리에서 p, default=2이므로 당연히 유클리디안 거리로 되어있다

#### 참고2
K는 보통 sqrt(n)을 선택하는게 일반적이며, 홀수를 채택한다

In [21]:
np.sqrt(len(fish))

12.609520212918492

minmax scale을 했기 때문에 거리차이가 각 feature마다 거리차이가 근사해졌을것이라 생각

#### 참고3
그리드 서치를 통해 유클리드 vs 맨해튼 vs 코싸인 거리 셋 중 최적값을 찾아본다
* "euclidean","manhatten","chebyshev","minkowski"

**분류 모델에 사용되는 주요 평가 지표:**

| 평가 지표           | 설명                                           |
|-----------------------|----------------------------------------------|
| `'accuracy'`          | 정확도 (Accuracy)                            |
| `'precision'`         | 정밀도 (Precision)                          |
| `'recall'`            | 재현율 (Recall)                             |
| `'f1'`                | F1 점수 (F1-Score)                         |
| `'roc_auc'`           | ROC AUC 점수 (Receiver Operating Characteristic Area Under the Curve) |
| `'average_precision'` | 평균 정밀도 (Average Precision)             |

**회귀 모델에 사용되는 주요 평가 지표:**

| 평가 지표                | 설명                                       |
|----------------------------|------------------------------------------|
| `'r2'`                     | R-squared (결정 계수)                    |
| `'neg_mean_squared_error'` | 음의 평균 제곱 오차 (Negative Mean Squared Error) |
| `'neg_mean_absolute_error'` | 음의 평균 절대 오차 (Negative Mean Absolute Error) |


In [32]:
import sklearn.metrics as skmet

In [33]:
def cos_dis(X,Y):
    return skmet.pairwise_distances(X,Y,metric="cosine")

In [50]:
params={'metric':["manhattan","euclidean"]}

In [51]:
grid_search=skmod.GridSearchCV(sknei.KNeighborsClassifier(n_neighbors=13),params,cv=5,error_score=0)

In [52]:
grid_search.fit(X=train[train.columns.difference(["Species"])],y=train["Species"])

GridSearchCV(cv=5, error_score=0,
             estimator=KNeighborsClassifier(n_neighbors=13),
             param_grid={'metric': ['manhattan', 'euclidean']})

In [56]:
print(grid_search.best_params_)
print(grid_search.best_score_)
# 디폴트가 accuaracy다

{'metric': 'euclidean'}
0.6855384615384617


-------

#### 참고 5. 코사인 거리
> 즉 2차원 데이터일 때 유용

In [54]:
def cos_dis(X,Y):
    return skmet.pairwise_distances(X,Y,metric='cosine')

---------

우리는 유클리디안 거리를 사용했을 때 가장 좋은 모델이 됨을 알았다.

### 예측해보기

In [57]:
best_knn=grid_search.best_estimator_

In [63]:
pd.merge(test[["Species"]],
         pd.DataFrame(best_knn.predict(test[test.columns.difference(["Species"])])).set_index(test.index).rename(columns={0:'predict'}),
         left_index=True,right_index=True)

,Species,predict
70,Parkki,Bream
27,Bream,Bream
147,Smelt,Smelt
131,Pike,Pike
100,Perch,Perch
132,Pike,Pike
82,Perch,Perch
15,Bream,Bream
148,Smelt,Smelt
44,Roach,Roach


#### 정확도 측정

In [64]:
best_knn.score(test[test.columns.difference(["Species"])],test["Species"])

0.71875

71%의 정답률이 나왔다